for guideon how to use xesmf regridding, see:

https://xesmf.readthedocs.io/en/latest/index.html

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf

%matplotlib inline

In [2]:
input_path = '/home/hgrosselindemann/Documents/TORI/'

velocity_fine = xr.open_dataset(input_path + 'velocity_raw_filtered.nc')
wind = xr.open_dataset(input_path + 'ERA5_10m_winds.nc')
wind = wind.rename({'longitude':'lon','latitude':'lat'})
ssh = xr.open_dataset(input_path + 'cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D_1679615605995.nc')
ssh = ssh.rename({'longitude':'lon','latitude':'lat'})

In [11]:
wind_new = xr.Dataset(
    {
        "lat": (["lat"], ssh.lat.data),
        "lon": (["lon"], ssh.lon.data)
}
)

In [13]:
regridder = xesmf.Regridder(wind,wind_new,'bilinear')
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_29x25_45x41.nc 
Reuse pre-computed weights? False 
Input grid shape:           (29, 25) 
Output grid shape:          (45, 41) 
Periodic in longitude?      False

In [14]:
wind_new = regridder(wind)

In [15]:
wind_new = wind_new.where(wind_new.u10!=0,drop=True)

In [23]:
wind_new.to_netcdf(input_path + 'ERA5_10m_wind_CMEMSGrid.nc')